In [ ]:
def import_functions():
    from pyspark.sql.functions import *
    from pyspark.sql.types import *
    import time
    from datetime import datetime
    import json
    import random

In [2]:
def create_spark_session():
    import pyspark
    import os
    from pyspark.sql import SparkSession
    spark=SparkSession.builder.appName('basic_stream').getOrCreate()
    os.environ['PYSPARK_PYTHON'] = "C:/Users/Administrator/AppData/Local/Programs/Python/Python311/python.exe"
    os.environ['PYSPARK_DRIVER_PYTHON'] = "C:/Users/Administrator/AppData/Local/Programs/Python/Python311/python.exe"
    
    return spark

In [3]:
def creat_stream_dataframe():
    static = spark.read.json(file_path)
    dataSchema = static.schema
    streaming_df = spark.readStream.schema(dataSchema).option("maxFilesPerTrigger", 1).json(file_path)
    spark.conf.set("spark.sql.shuffle.partitions", 1)
    return streaming_df

In [4]:
def get_activity_count():
    activityCounts = streaming_df.groupBy("gt").count()
    activityQuery = activityCounts.writeStream.queryName("activity_count")\
        .format("memory").outputMode("complete")\
        .start()

    for x in range(5):
        spark.sql('SELECT * FROM activity_count').show()
        time.sleep(10)

    activityQuery.awaitTermination()

In [5]:
def get_user_count():
    userCounts = streaming_df.groupBy('User').count()
    userQuery = userCounts.writeStream.queryName("user_count")\
        .format("memory").outputMode("complete")\
        .start()
    for x in range(5):
        spark.sql('SELECT * FROM user_count').show()
        time.sleep(10)

    userQuery.awaitTermination()

In [ ]:
# import_functions()
spark = create_spark_session()
file_path = r'E:\ABD\PySpark\DataSet\activity-data'
streaming_df = creat_stream_dataframe()
get_activity_count()
# get_user_count()

+---+-----+
| gt|count|
+---+-----+
+---+-----+

+----------+------+
|        gt| count|
+----------+------+
|  stairsup| 94067|
|     stand|102464|
|       sit|110778|
|      walk|119304|
|      bike| 97175|
|      null| 94033|
|stairsdown| 84286|
+----------+------+

+----------+------+
|        gt| count|
+----------+------+
|  stairsup|156800|
|     stand|170778|
|       sit|184620|
|      walk|198839|
|      bike|161965|
|      null|156721|
|stairsdown|140456|
+----------+------+



In [1]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
import time
from datetime import datetime
import json
import random